# Add anubis_return in the analysis
***

In [ ]:
import os 
import pandas as pd 
import numpy as np 

path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"

> Open DF and create an index sequence column "anubis_return"

In [ ]:
# Open : 
anubis_return = pd.read_csv(f"{path_work}/Anubis_return.predictions.0709.big.tsv", sep = "\t", names = ["protein_name", "start", "end", "domain_seq", "protein_seq"])
anubis_return_unique = anubis_return.drop_duplicates(subset = ["protein_seq"], keep = "first", ignore_index = True)
dico_index = {row["protein_seq"] : index for index,row in anubis_return_unique.iterrows()}
anubis_return["index_dpo"] = anubis_return["protein_seq"].apply(lambda x : f"anubis_return__{dico_index[x]}")

# Write : 
anubis_return.to_csv(f"{path_work}/Anubis_return.predictions.0709.big.annotated.tsv", sep = "\t", header = 0, index = False)

> Write the multi fasta file 

In [ ]:
with open(f"{path_work}/Anubis_return.multi.fasta" , "w") as outfile : 
    for index, row in anubis_return_unique.iterrows() : 
        outfile.write(f">anubis_return__{index}\n{row['domain_seq']}\n")

>ESM2 predictions 

In [ ]:
#!/bin/bash
#BATCH --job-name=ESM_2__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40 
#SBATCH --mem=100gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=ESM_2__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate embeddings

python /home/conchae/software/esm/scripts/extract.py \
esm2_t33_650M_UR50D \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Anubis_return.multi.fasta \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Anubis_return.multi.fasta.esm_out \
--repr_layers 33 \
--include mean per_tok


In [ ]:
import torch
import os 
import pandas as pd
from tqdm import tqdm 


path_esm = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Anubis_return.multi.fasta.esm_out"

embeddings_esm = {}
for file in tqdm(os.listdir(path_esm)) :
    path_file = f"{path_esm}/{file}"
    index = file.split(".")[0]
    embb = torch.load(f"{path_file}")["mean_representations"][33].tolist()
    embeddings_esm[index] = embb
    
with open(f"/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/anubis_return.esm2.embedding.csv" , "w") as outfile :
    for index in tqdm(embeddings_esm) :
        outfile.write(f"{index},")
        for _,  emb in enumerate(embeddings_esm[index]) :
            outfile.write(f"{emb},")
        outfile.write("\n")

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/anubis_return.esm2.embedding.csv \
/media/concha-eloko/Linux/PPT_clean

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Anubis_return.predictions.0709.big.annotated.tsv \
/media/concha-eloko/Linux/PPT_clean
